In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [113]:
summoner_name = "쪼렙이다말로하자"

match = pd.read_json("data/data_summoner_match_data_{name}.json".format(name=summoner_name))
timeline = pd.read_json("data/data_summoner_match_timeline_data_{name}.json".format(name=summoner_name))

# 경기 관련 데이터

In [114]:
match_length = len(match["info"])
match_length

200

In [115]:
info = match["info"]

participants = []
teams = []
for i in range(match_length):
    participants.append(info[i]["participants"])
    teams.append(info[i]["teams"])
    del(info[i]["participants"])
    del(info[i]["teams"])

In [116]:
df_info = pd.json_normalize(info)
df_info_1 = df_info[["gameDuration", "gameMode", "gameType", "gameVersion", "mapId"]]
display(df_info_1.head(10))

,gameDuration,gameMode,gameType,gameVersion,mapId
0,1371736,CLASSIC,MATCHED_GAME,11.19.398.9466,11
1,1542991,CLASSIC,MATCHED_GAME,11.19.398.9466,11
2,1468270,CLASSIC,MATCHED_GAME,11.19.398.9466,11
3,1963800,CLASSIC,MATCHED_GAME,11.19.398.9466,11
4,1634952,CLASSIC,MATCHED_GAME,11.19.398.9466,11
5,1932577,CLASSIC,MATCHED_GAME,11.19.398.9466,11
6,1478309,CLASSIC,MATCHED_GAME,11.19.398.9466,11
7,1207067,CLASSIC,MATCHED_GAME,11.19.398.2521,11
8,914139,CLASSIC,MATCHED_GAME,11.19.398.2521,11
9,1516101,CLASSIC,MATCHED_GAME,11.18.395.7538,11


In [117]:
df_participants = pd.json_normalize(participants)
display(df_participants.head(1))

,0,1,2,3,4,5,6,7,8,9
0,"{'assists': 3, 'baronKills': 0, 'bountyLevel':...","{'assists': 8, 'baronKills': 1, 'bountyLevel':...","{'assists': 8, 'baronKills': 0, 'bountyLevel':...","{'assists': 9, 'baronKills': 0, 'bountyLevel':...","{'assists': 9, 'baronKills': 0, 'bountyLevel':...","{'assists': 2, 'baronKills': 0, 'bountyLevel':...","{'assists': 4, 'baronKills': 0, 'bountyLevel':...","{'assists': 3, 'baronKills': 0, 'bountyLevel':...","{'assists': 2, 'baronKills': 0, 'bountyLevel':...","{'assists': 4, 'baronKills': 0, 'bountyLevel':..."


In [118]:
df_teams = pd.json_normalize(teams)
df_teams.columns = ["100", "200"]   # teamId
display(df_teams.head(5))

,100,200
0,"{'bans': [{'championId': -1, 'pickTurn': 1}, {...","{'bans': [{'championId': -1, 'pickTurn': 6}, {..."
1,"{'bans': [{'championId': 238, 'pickTurn': 1}, ...","{'bans': [{'championId': 126, 'pickTurn': 6}, ..."
2,"{'bans': [{'championId': 23, 'pickTurn': 1}, {...","{'bans': [{'championId': 39, 'pickTurn': 6}, {..."
3,"{'bans': [{'championId': 412, 'pickTurn': 1}, ...","{'bans': [{'championId': 412, 'pickTurn': 6}, ..."
4,"{'bans': [{'championId': 711, 'pickTurn': 1}, ...","{'bans': [{'championId': 64, 'pickTurn': 6}, {..."


In [119]:
df_info_1.to_csv("data/{}/info.csv".format(summoner_name))
df_participants.to_csv("data/{}/participants.csv".format(summoner_name))
df_teams.to_csv("data/{}/teams.csv".format(summoner_name))

# 인베이드 관련 데이터

In [120]:
match_invade = 0

## 이벤트 분석

In [121]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    if len(events) < 3: 
        print("다시하기 발생!")
        continue    # 게임이 3분 이내로 끝났을 경우를 제외 
    for t in range(1,3):  # 1 ~ 2 분 사이
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1

    if is_invade[match_index]:
        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        
        # use df_teams
        invade_win_team[match_index] = 100 if df_teams["100"][match_index]["win"] else 200
        
        print("--------------------------------------------------")
        print("invade detected: %d" % match_index)
        print("timestamp(minutes): %.2f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% ("blue" if invade_win_team[match_index] == 100 else "red"))

--------------------------------------------------
invade detected: 3
timestamp(minutes): 1.87
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 4419 , red: 5324
< gold_gap >
  -905
< win >
  blue
--------------------------------------------------
invade detected: 8
timestamp(minutes): 1.59
< kills >
  blue : 3 , red : 0
< total_gold_3_min >
  blue: 5579 , red: 4008
< gold_gap >
  1571
< win >
  blue
--------------------------------------------------
invade detected: 9
timestamp(minutes): 1.75
< kills >
  blue : 2 , red : 1
< total_gold_3_min >
  blue: 5063 , red: 4624
< gold_gap >
  439
< win >
  blue
--------------------------------------------------
invade detected: 13
timestamp(minutes): 1.56
< kills >
  blue : 1 , red : 0
< total_gold_3_min >
  blue: 4798 , red: 4289
< gold_gap >
  509
< win >
  blue
--------------------------------------------------
invade detected: 17
timestamp(minutes): 1.10
< kills >
  blue : 2 , red : 0
< total_gold_3_min >
  blue: 5910 , red: 4207


In [122]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap,
    df_info_1["gameMode"]]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap",
    "gameMode"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap,gameMode
8,True,95344,3,0,100,5579,4008,1571,CLASSIC
9,True,105263,2,1,100,5063,4624,439,CLASSIC
13,True,93590,1,0,100,4798,4289,509,CLASSIC
17,True,66227,2,0,100,5910,4207,1703,CLASSIC
27,True,103481,1,0,100,4908,4408,500,CLASSIC
36,True,61481,1,0,200,4858,4474,384,CLASSIC
43,True,98819,2,0,100,5600,4562,1038,CLASSIC
44,True,101375,1,0,100,5365,3741,1624,CLASSIC
47,True,112460,1,0,100,4496,3976,520,CLASSIC
48,True,54482,1,0,100,5442,4641,801,CLASSIC


77.78%


In [123]:
match_invade

46

In [124]:
df_invade.to_csv("data/{}/invade.csv".format(summoner_name))